In [28]:
"""
    This script walks through the various steps to train and use CLMBR.

    In order to use this script, the assumption is that you already have a set of labels and an extract
"""

import os
import pickle
import re
import pandas as pd
import shutil

#EXTRACT_LOCATION = "/local-scratch/nigam/projects/ethanid/shared_tutorial_files/piton_new3_extract"
EXTRACT_LOCATION = "/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/som-nero-phi-nigam-starr.shahlab_omop_cdm5_subset_2023_05_06_extract_no_observation_v2"
LABELS = "/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/Allmortality_100000label_0to12m/labeled_patients.pkl"
TEMP_STORAGE = "trash/clmbr_data"

os.makedirs(TEMP_STORAGE, exist_ok=True)

In [9]:
!export CUDA_VISIBLE_DEVICES=6

In [10]:
"""
The first step of training CLMBR is creating a dictionary, that helps map codes to integers that can be used within a neural network.
"""

DICTIONARY_PATH = os.path.join(TEMP_STORAGE, "dictionary")

if not os.path.exists(DICTIONARY_PATH):
    assert 0 == os.system(f"clmbr_create_dictionary {DICTIONARY_PATH} --data_path {EXTRACT_LOCATION}")

In [11]:
"""
The second step of training CLMBR is to prepare the batches that will actually get fed into the neural network.
"""

CLMBR_BATCHES = os.path.join(TEMP_STORAGE, "clmbr_batches")

if not os.path.exists(CLMBR_BATCHES):
    assert 0 == os.system(
        f"clmbr_create_batches {CLMBR_BATCHES} --data_path {EXTRACT_LOCATION} --dictionary {DICTIONARY_PATH} --task clmbr"
    )

2023-05-21 07:19:26,291 [MainThread  ] [INFO ]  Preparing batches with Namespace(directory='trash/clmbr_data/clmbr_batches', data_path='/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/som-nero-phi-nigam-starr.shahlab_omop_cdm5_subset_2023_05_06_extract_no_observation_v2', dictionary_path='trash/clmbr_data/dictionary', task='clmbr', transformer_vocab_size=65536, clmbr_survival_dictionary_path=None, labeled_patients_path=None, is_hierarchical=False, seed=97, val_start=80, test_start=85, batch_size=16384, note_embedding_data=None, limit_to_patients_file=None, limit_before_date=None)
2023-05-21 07:20:11,831 [MainThread  ] [INFO ]  Wrote config ...
2023-05-21 07:20:11,831 [MainThread  ] [INFO ]  Starting to load


When mapping codes, dropped When mapping codes, dropped 0 out of 65536When mapping codes, dropped 0 out of 65536When mapping codes, dropped 0 out of 655360 out of 65536



When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0When mapping codes, dropped 0 out of 65536When mapping codes, dropped 0When mapping codes, dropped When mapping codes, dropped 00When mapping codes, dropped  out of 
65536 out of 
65536
When mapping codes, dropped When mapping codes, dropped 0 out of 65536
 out of 65536
0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of When mapping codes, dropped 0 out of 655360 out of 65536
65536

 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mappin

2023-05-21 07:22:14,257 [MainThread  ] [INFO ]  Loaded


When mapping codes, dropped 0 out of 65536


2023-05-21 07:23:21,919 [MainThread  ] [INFO ]  Number of train patients 98177


In [12]:
"""
Given the batches, it is now possible to train CLMBR. By default it will train for 100 epochs, with early stopping.
"""

MODEL_PATH = os.path.join(TEMP_STORAGE, "clmbr_model")

if not os.path.exists(MODEL_PATH):
    assert 0 == os.system(
        f"clmbr_train_model {MODEL_PATH} --data_path {EXTRACT_LOCATION} --batches_path {CLMBR_BATCHES} --learning_rate 1e-4 --rotary_type per_head --num_batch_threads 3 --max_iter 1000"
    )

2023-05-21 07:23:26,982 [MainThread  ] [INFO ]  Training model with Namespace(directory='trash/clmbr_data/clmbr_model', data_path='/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/som-nero-phi-nigam-starr.shahlab_omop_cdm5_subset_2023_05_06_extract_no_observation_v2', batches_path='trash/clmbr_data/clmbr_batches', learning_rate=0.0001, rotary_type='per_head', clmbr_survival_dim=None, num_batch_threads=3, start_from_checkpoint=None, freeze_weights=False, token_dropout=0, internal_dropout=0, weight_decay=0, max_iter=1000, hidden_size=768, intermediate_size=3072, n_heads=12, n_layers=6, attention_width=512, early_stopping_window_steps=None)
2023-05-21 07:23:42,942 [MainThread  ] [INFO ]  Got config {'data_path': '/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/som-nero-phi-nigam-starr.shahlab_omop_cdm5_subset_2023_05_06_extract_no_observation_v2', 'batch_info_path': 'trash/clmbr_data/clmbr_batches/batch_info.msgpack', 'seed': 97, 'task': {'type': 'clmbr', 'vocab_size': 

When mapping codes, dropped 0 out of 65536


2023-05-21 07:24:50,706 [MainThread  ] [INFO ]  Loaded batches 98177 6048
2023-05-21 07:24:52,349 [MainThread  ] [INFO ]  Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: CUDA Interpreter Host
2023-05-21 07:24:52,349 [MainThread  ] [INFO ]  Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'
2023-05-21 07:24:52,350 [MainThread  ] [INFO ]  Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.
2023-05-21 07:24:53,340 [MainThread  ] [INFO ]  Got dummy batch {'num_indices': ((), dtype('int32'), StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)), 'num_patients': ((), dtype('int32'), StreamExecutorGpuDevice(id=0, process_index=0, slice_index=0)), 'offsets': ((512,), dtype('uint32

Compiling the transformer ... (16384,) (16384,)


2023-05-21 07:24:55,857 [MainThread  ] [INFO ]  Done initing {'EHRTransformer/~/CLMBRTask/~/linear': {'b': ((8192,), dtype('float32')), 'w': ((768, 8192), dtype('float32'))}, 'EHRTransformer/~/TransformerFeaturizer/~/Transformer/~/embed': {'embeddings': ((65536, 768), dtype('float32'))}, 'EHRTransformer/~/TransformerFeaturizer/~/Transformer/~/loop_0/TransformerBlock/~/linear': {'b': ((5376,), dtype('float32')), 'w': ((768, 5376), dtype('float32'))}, 'EHRTransformer/~/TransformerFeaturizer/~/Transformer/~/loop_0/TransformerBlock/~/linear_1': {'b': ((768,), dtype('float32')), 'w': ((3840, 768), dtype('float32'))}, 'EHRTransformer/~/TransformerFeaturizer/~/Transformer/~/loop_0/TransformerBlock/~/rms_norm': {'scale': ((768,), dtype('float32'))}, 'EHRTransformer/~/TransformerFeaturizer/~/Transformer/~/loop_1/TransformerBlock/~/linear': {'b': ((5376,), dtype('float32')), 'w': ((768, 5376), dtype('float32'))}, 'EHRTransformer/~/TransformerFeaturizer/~/Transformer/~/loop_1/TransformerBlock/~/l

Compiling the transformer ... (16384,) (16384,)


2023-05-21 07:25:02,161 [MainThread  ] [INFO ]  Starting train loss {'loss': 9.388360977172852, 'c_statistic': -9.388360977172852}
2023-05-21 07:25:05,119 [MainThread  ] [INFO ]  Starting dev loss {'loss': 9.388991355895996, 'c_statistic': -9.388991355895996}
Working with seed 9700


When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
Compiling the transformer ... (16384,) (16384,)


/home/zphuo/miniconda3/envs/FEMR_ENV/lib/python3.10/site-packages/jax/_src/interpreters/mlir.py:711: UserWarning: Some donated buffers were not usable: ShapedArray(uint32[16384]), ShapedArray(float32[16384]), ShapedArray(uint32[16384]), ShapedArray(int32[], weak_type=True), ShapedArray(uint32[16384]), ShapedArray(bool[16384]).
See an explanation at https://jax.readthedocs.io/en/latest/faq.html#buffer-donation.
  warnings.warn(f"Some donated buffers were not usable: {', '.join(unused_donations)}.\n{msg}")
2023-05-21 07:29:54,264 [MainThread  ] [INFO ]  [Step 0]
2023-05-21 07:30:13,175 [MainThread  ] [INFO ]  [Step 100]
2023-05-21 07:30:32,224 [MainThread  ] [INFO ]  [Step 200]
2023-05-21 07:30:51,322 [MainThread  ] [INFO ]  [Step 300]
2023-05-21 07:31:10,461 [MainThread  ] [INFO ]  [Step 400]
2023-05-21 07:31:29,755 [MainThread  ] [INFO ]  [Step 500]
2023-05-21 07:31:29,756 [MainThread  ] [INFO ]  Loss scale DynamicLossScale(loss_scale=Array(32768., dtype=float32), counter=Array(500, dt

# customized task batch (3 mortality 1 readmission)

In [24]:
# read labeledpatients and downselect to PE cohort and dump to a new pickle file
def downselect(labeledpatients_path, cohort_set):
    
    with open(labeledpatients_path, "rb") as LABELS:
        labeledpatients = pickle.load(LABELS)
    print('before downselect', len(labeledpatients), 'patients')
          
    for i, n in enumerate(labeledpatients.get_all_patient_ids()):
        if n not in cohort_set:
            labeledpatients.__delitem__(n)
            
    labeledpatients_path_new = labeledpatients_path[:-4] + "_PE.pkl"

    with open(labeledpatients_path_new, "wb") as LABELS:
        pickle.dump(labeledpatients, LABELS)
    print('after downselect', len(labeledpatients), 'patients')
    return labeledpatients_path_new

In [25]:
cohort_csv='/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/som-nero-phi-nigam-starr.frazier/radfusion3_cohort.csv'

ALLPE_file = cohort_csv
ALLPE_df = pd.read_csv(ALLPE_file)
PE_ids = set(ALLPE_df.person_id)


cohort='PE'

In [29]:
# create a task batch for labeled patients
#12 month
import os
TEMP_STORAGE = '/local-scratch/nigam/projects/zphuo/models/clinical-text-with-codes/clmbr_lr_1e-05_wd_0.0_id_0.0_td_0.0_rt_global_maxiter_10000000_hs_768_is_3072_nh_12_nl_12_aw_512_converted_dict/tasks/12_month_mortality/100_percent/'
DICTIONARY_PATH = re.sub(r'tasks/.+', 'dictionary', TEMP_STORAGE)


if cohort == 'PE':
    TASK_BATCHES = os.path.join(TEMP_STORAGE, f"task_batches_PE")   
    LABELS = "/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/Allmortality_100000label_0to12m/labeled_patients.pkl"
    LABELS = downselect(LABELS, PE_ids)
else:
    TASK_BATCHES = os.path.join(TEMP_STORAGE, f"task_batches")   
    LABELS = "/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/Allmortality_100000label_0to12m/labeled_patients.pkl"


if not os.path.exists(TASK_BATCHES):
    assert 0 == os.system(
        f"clmbr_create_batches {TASK_BATCHES} --data_path {EXTRACT_LOCATION} --dictionary {DICTIONARY_PATH} --task labeled_patients --labeled_patients_path {LABELS} --val_start 80"
    )
else:
    shutil.rmtree(TASK_BATCHES, ignore_errors=True)
    assert 0 == os.system(
        f"clmbr_create_batches {TASK_BATCHES} --data_path {EXTRACT_LOCATION} --dictionary {DICTIONARY_PATH} --task labeled_patients --labeled_patients_path {LABELS} --val_start 80"
    )

before downselect 3686202 patients
after downselect 20192 patients


2023-05-21 07:57:06,367 [MainThread  ] [INFO ]  Preparing batches with Namespace(directory='/local-scratch/nigam/projects/zphuo/models/clinical-text-with-codes/clmbr_lr_1e-05_wd_0.0_id_0.0_td_0.0_rt_global_maxiter_10000000_hs_768_is_3072_nh_12_nl_12_aw_512_converted_dict/tasks/12_month_mortality/100_percent/task_batches_PE', data_path='/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/som-nero-phi-nigam-starr.shahlab_omop_cdm5_subset_2023_05_06_extract_no_observation_v2', dictionary_path='/local-scratch/nigam/projects/zphuo/models/clinical-text-with-codes/clmbr_lr_1e-05_wd_0.0_id_0.0_td_0.0_rt_global_maxiter_10000000_hs_768_is_3072_nh_12_nl_12_aw_512_converted_dict/dictionary', task='labeled_patients', transformer_vocab_size=65536, clmbr_survival_dictionary_path=None, labeled_patients_path='/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/Allmortality_100000label_0to12m/labeled_patients_PE.pkl', is_hierarchical=False, seed=97, val_start=80, test_start=85, batch_size=16

When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
 out of 65536
When mapping codes, dropped 0 out of 65536When mapping codes, dropped When mapping codes, dropped 0 out of 65536
0 out of 65536

When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536When mapping codes, dropped 0
 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0When mapping codes, dropped When mapping codes, dropped 0 out of 655

2023-05-21 07:59:22,260 [MainThread  ] [INFO ]  Loaded


When mapping codes, dropped 0 out of 65536


2023-05-21 08:00:26,071 [MainThread  ] [INFO ]  Number of train patients 17232


In [30]:
# create a task batch for labeled patients
#6 month
import os
TEMP_STORAGE = '/local-scratch/nigam/projects/zphuo/models/clinical-text-with-codes/clmbr_lr_1e-05_wd_0.0_id_0.0_td_0.0_rt_global_maxiter_10000000_hs_768_is_3072_nh_12_nl_12_aw_512_converted_dict/tasks/6_month_mortality/100_percent/'
DICTIONARY_PATH = re.sub(r'tasks/.+', 'dictionary', TEMP_STORAGE)


if cohort == 'PE':
    TASK_BATCHES = os.path.join(TEMP_STORAGE, f"task_batches_PE")
    LABELS = "/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/Allmortality_100000label_0to6m/labeled_patients.pkl"
    LABELS = downselect(LABELS, PE_ids)
else:
    TASK_BATCHES = os.path.join(TEMP_STORAGE, f"task_batches")
    LABELS = "/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/Allmortality_100000label_0to6m/labeled_patients.pkl"
    

if not os.path.exists(TASK_BATCHES):
    assert 0 == os.system(
        f"clmbr_create_batches {TASK_BATCHES} --data_path {EXTRACT_LOCATION} --dictionary {DICTIONARY_PATH} --task labeled_patients --labeled_patients_path {LABELS} --val_start 80"
    )
else:
    shutil.rmtree(TASK_BATCHES, ignore_errors=True)
    assert 0 == os.system(
        f"clmbr_create_batches {TASK_BATCHES} --data_path {EXTRACT_LOCATION} --dictionary {DICTIONARY_PATH} --task labeled_patients --labeled_patients_path {LABELS} --val_start 80"
    )

before downselect 3686202 patients
after downselect 20192 patients


2023-05-21 08:00:45,560 [MainThread  ] [INFO ]  Preparing batches with Namespace(directory='/local-scratch/nigam/projects/zphuo/models/clinical-text-with-codes/clmbr_lr_1e-05_wd_0.0_id_0.0_td_0.0_rt_global_maxiter_10000000_hs_768_is_3072_nh_12_nl_12_aw_512_converted_dict/tasks/6_month_mortality/100_percent/task_batches_PE', data_path='/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/som-nero-phi-nigam-starr.shahlab_omop_cdm5_subset_2023_05_06_extract_no_observation_v2', dictionary_path='/local-scratch/nigam/projects/zphuo/models/clinical-text-with-codes/clmbr_lr_1e-05_wd_0.0_id_0.0_td_0.0_rt_global_maxiter_10000000_hs_768_is_3072_nh_12_nl_12_aw_512_converted_dict/dictionary', task='labeled_patients', transformer_vocab_size=65536, clmbr_survival_dictionary_path=None, labeled_patients_path='/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/Allmortality_100000label_0to6m/labeled_patients_PE.pkl', is_hierarchical=False, seed=97, val_start=80, test_start=85, batch_size=1638

When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 655360 out of 65536

When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0When mapping codes, dropped 0 out of 65536
 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0When mapping codes, dropped 0 out of 65536 out of 65536

When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mappin

2023-05-21 08:03:05,653 [MainThread  ] [INFO ]  Loaded


When mapping codes, dropped 0 out of 65536


2023-05-21 08:04:11,274 [MainThread  ] [INFO ]  Number of train patients 18295


In [31]:
# create a task batch for labeled patients
#3 month
import os
TEMP_STORAGE = '/local-scratch/nigam/projects/zphuo/models/clinical-text-with-codes/clmbr_lr_1e-05_wd_0.0_id_0.0_td_0.0_rt_global_maxiter_10000000_hs_768_is_3072_nh_12_nl_12_aw_512_converted_dict/tasks/3_month_mortality/100_percent/'
DICTIONARY_PATH = re.sub(r'tasks/.+', 'dictionary', TEMP_STORAGE)


if cohort == 'PE':
    TASK_BATCHES = os.path.join(TEMP_STORAGE, "task_batches_PE")
    LABELS = "/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/Allmortality_100000label_0to3m/labeled_patients.pkl"
    LABELS = downselect(LABELS, PE_ids)
else:
    TASK_BATCHES = os.path.join(TEMP_STORAGE, "task_batches")
    LABELS = "/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/Allmortality_100000label_0to3m/labeled_patients.pkl"

if not os.path.exists(TASK_BATCHES):
    assert 0 == os.system(
        f"clmbr_create_batches {TASK_BATCHES} --data_path {EXTRACT_LOCATION} --dictionary {DICTIONARY_PATH} --task labeled_patients --labeled_patients_path {LABELS} --val_start 80"
    )
else:
    shutil.rmtree(TASK_BATCHES, ignore_errors=True)
    assert 0 == os.system(
        f"clmbr_create_batches {TASK_BATCHES} --data_path {EXTRACT_LOCATION} --dictionary {DICTIONARY_PATH} --task labeled_patients --labeled_patients_path {LABELS} --val_start 80"
    )

before downselect 3686202 patients
after downselect 20192 patients


2023-05-21 08:04:32,575 [MainThread  ] [INFO ]  Preparing batches with Namespace(directory='/local-scratch/nigam/projects/zphuo/models/clinical-text-with-codes/clmbr_lr_1e-05_wd_0.0_id_0.0_td_0.0_rt_global_maxiter_10000000_hs_768_is_3072_nh_12_nl_12_aw_512_converted_dict/tasks/3_month_mortality/100_percent/task_batches_PE', data_path='/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/som-nero-phi-nigam-starr.shahlab_omop_cdm5_subset_2023_05_06_extract_no_observation_v2', dictionary_path='/local-scratch/nigam/projects/zphuo/models/clinical-text-with-codes/clmbr_lr_1e-05_wd_0.0_id_0.0_td_0.0_rt_global_maxiter_10000000_hs_768_is_3072_nh_12_nl_12_aw_512_converted_dict/dictionary', task='labeled_patients', transformer_vocab_size=65536, clmbr_survival_dictionary_path=None, labeled_patients_path='/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/Allmortality_100000label_0to3m/labeled_patients_PE.pkl', is_hierarchical=False, seed=97, val_start=80, test_start=85, batch_size=1638

When mapping codes, dropped When mapping codes, dropped When mapping codes, dropped 0 out of 65536
When mapping codes, dropped When mapping codes, dropped When mapping codes, dropped 0 out of 65536When mapping codes, dropped When mapping codes, dropped When mapping codes, dropped 00When mapping codes, dropped  out of When mapping codes, dropped 0 out of 655360 out of 065536 out of 
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0When mapping codes, dropped 0 out of 65536
0 out of 65536
65536
0 out of 65536
 out of 65536
 out of 65536When mapping codes, dropped 00 out of  out of 065536
 out of 65536
65536When mapping codes, dropped 

When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536

When mapping codes, dropped 0 out

2023-05-21 08:06:54,349 [MainThread  ] [INFO ]  Loaded


When mapping codes, dropped 0 out of 65536


2023-05-21 08:07:59,112 [MainThread  ] [INFO ]  Number of train patients 18846


In [32]:
# create a task batch for labeled patients
#30 day
import os
TEMP_STORAGE = '/local-scratch/nigam/projects/zphuo/models/clinical-text-with-codes/clmbr_lr_1e-05_wd_0.0_id_0.0_td_0.0_rt_global_maxiter_10000000_hs_768_is_3072_nh_12_nl_12_aw_512_converted_dict/tasks/30_day_readmission/100_percent/'
DICTIONARY_PATH = re.sub(r'tasks/.+', 'dictionary', TEMP_STORAGE)

if cohort == 'PE':
    TASK_BATCHES = os.path.join(TEMP_STORAGE, "task_batches_PE")
    LABELS = "/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/readmission_100000label_0to30d/labeled_patients.pkl"
    LABELS = downselect(LABELS, PE_ids)
else:
    TASK_BATCHES = os.path.join(TEMP_STORAGE, "task_batches")
    LABELS = "/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/readmission_100000label_0to30d/labeled_patients.pkl"


if not os.path.exists(TASK_BATCHES):
    assert 0 == os.system(
        f"clmbr_create_batches {TASK_BATCHES} --data_path {EXTRACT_LOCATION} --dictionary {DICTIONARY_PATH} --task labeled_patients --labeled_patients_path {LABELS} --val_start 80"
    )
else:
    shutil.rmtree(TASK_BATCHES, ignore_errors=True)
    assert 0 == os.system(
        f"clmbr_create_batches {TASK_BATCHES} --data_path {EXTRACT_LOCATION} --dictionary {DICTIONARY_PATH} --task labeled_patients --labeled_patients_path {LABELS} --val_start 80"
    )

before downselect 3686202 patients
after downselect 20192 patients


2023-05-21 08:08:16,904 [MainThread  ] [INFO ]  Preparing batches with Namespace(directory='/local-scratch/nigam/projects/zphuo/models/clinical-text-with-codes/clmbr_lr_1e-05_wd_0.0_id_0.0_td_0.0_rt_global_maxiter_10000000_hs_768_is_3072_nh_12_nl_12_aw_512_converted_dict/tasks/30_day_readmission/100_percent/task_batches_PE', data_path='/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/som-nero-phi-nigam-starr.shahlab_omop_cdm5_subset_2023_05_06_extract_no_observation_v2', dictionary_path='/local-scratch/nigam/projects/zphuo/models/clinical-text-with-codes/clmbr_lr_1e-05_wd_0.0_id_0.0_td_0.0_rt_global_maxiter_10000000_hs_768_is_3072_nh_12_nl_12_aw_512_converted_dict/dictionary', task='labeled_patients', transformer_vocab_size=65536, clmbr_survival_dictionary_path=None, labeled_patients_path='/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/readmission_100000label_0to30d/labeled_patients_PE.pkl', is_hierarchical=False, seed=97, val_start=80, test_start=85, batch_size=163

When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of When mapping codes, dropped When mapping codes, dropped 0065536 out of 65536
 out of 65536

When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mappin

2023-05-21 08:11:02,086 [MainThread  ] [INFO ]  Loaded


When mapping codes, dropped 0 out of 65536


2023-05-21 08:12:19,002 [MainThread  ] [INFO ]  Number of train patients 3688


In [19]:
"""
You now have a complete CLMBR model. It is now time to generate representations for your task of interest.

The first step of doing so is to generate batches for that task.
"""

TASK_BATCHES = os.path.join(TEMP_STORAGE, "task_batches")

if not os.path.exists(TASK_BATCHES):
    assert 0 == os.system(
        f"clmbr_create_batches {TASK_BATCHES} --data_path {EXTRACT_LOCATION} --dictionary {DICTIONARY_PATH} --task labeled_patients --labeled_patients_path {LABELS}"
    )

REPRESENTATIONS = os.path.join(TEMP_STORAGE, "clmbr_reprs")



2023-05-21 07:36:11,469 [MainThread  ] [INFO ]  Preparing batches with Namespace(directory='/local-scratch/nigam/projects/zphuo/models/clinical-text-with-codes/clmbr_lr_1e-05_wd_0.0_id_0.0_td_0.0_rt_global_maxiter_10000000_hs_768_is_3072_nh_12_nl_12_aw_512_converted_dict/tasks/30_day_readmission/100_percent/task_batches', data_path='/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/som-nero-phi-nigam-starr.shahlab_omop_cdm5_subset_2023_05_06_extract_no_observation_v2', dictionary_path='/local-scratch/nigam/projects/zphuo/models/clinical-text-with-codes/clmbr_lr_1e-05_wd_0.0_id_0.0_td_0.0_rt_global_maxiter_10000000_hs_768_is_3072_nh_12_nl_12_aw_512_converted_dict/dictionary', task='labeled_patients', transformer_vocab_size=65536, clmbr_survival_dictionary_path=None, labeled_patients_path='/local-scratch/nigam/projects/zphuo/data/omop_extract_PHI/readmission_100000label_0to30d/labeled_patients_PE.pkl', is_hierarchical=False, seed=97, val_start=80, test_start=85, batch_size=16384,

When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0When mapping codes, dropped  out of 65536When mapping codes, dropped 0 out of 65536

0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mapping codes, dropped 0 out of 65536
When mappin

2023-05-21 07:38:29,072 [MainThread  ] [INFO ]  Loaded


When mapping codes, dropped 0 out of 65536


2023-05-21 07:39:36,491 [MainThread  ] [INFO ]  Number of train patients 3688


In [20]:
"""
Finally, you can generate features for that generated batches.
"""

if not os.path.exists(REPRESENTATIONS):
    assert 0 == os.system(
        f"clmbr_compute_representations {REPRESENTATIONS} --data_path {EXTRACT_LOCATION} --batches_path {TASK_BATCHES} --model_dir {MODEL_PATH}"
    )



When mapping codes, dropped 0 out of 65536
Compiling the transformer ... (16384,) (256,)


In [21]:
"""
Open the resulting representations and take a look at the data matrix.
"""

with open(REPRESENTATIONS, "rb") as f:
    reprs = pickle.load(f)

    print(reprs.keys())

    print("Pulling the data for the first label")
    print("Patient id", reprs["patient_ids"][0])
    print("Label time", reprs["labeling_time"][0])
    print("Representation", reprs["data_matrix"][0, :16])

dict_keys(['data_path', 'model', 'data_matrix', 'patient_ids', 'labeling_time'])
Pulling the data for the first label
Patient id 41487
Label time 2008-12-30 11:20:00
Representation [-0.2615  -1.416   -1.52     0.10803  1.909   -1.808    0.692   -0.271
 -0.723    1.358   -0.8765   0.759   -0.2634  -0.3965  -1.166    0.4863 ]


In [22]:
import pickle 

top_k_pickle = '/local-scratch/nigam/projects/rthapa84/data/MLHC/clmbr_dictionary_stats/code_only/top_100_texts.pickle'

with open(top_k_pickle, 'rb') as f:
    top_k = pickle.load(f)

In [23]:
top_k


{'  This encounter was opened erroneously.  ',
 '  This office note has been dictated.  Please see dictated note for details.      MRN:  [000000]  Visit Number: [000000]  ',
 '  This office note has been dictated.  Please see dictated note for details.      MRN:  [000000]  Visit Number: [000000]    Allergies, Medication Reconciliation, and History have been reviewed with the Patient.  Please see dictated note for details.    ',
 '(-)',
 '0 - 2',
 '0 - 3',
 '0-2',
 '0-3',
 '0-rare',
 '1+',
 '2+',
 '3+',
 '6-10',
 'A Positive',
 'Automated urine microscopic exam performed.',
 'Bw6',
 'COVID NOT DETECTED',
 'Clear',
 'Colorless',
 'Comment',
 'Completed',
 'DETECTED',
 'Detected',
 'Digital Photos',
 "Disclaimer: Molecular diagnostic test results should be interpreted in the context of standard clinical, laboratory and pathological findings. Molecular genetic test results impart a probabilistic risk of disease. In order to derive the most meaningful benefit from this testing, it is recomm